# Take-home problem Lab1, implementing a 2D convolution

I believe anyone can find a large number of resources for this specific topic, please go through any of them to get a hang of the actual theory. I definitely recommend this [TDS](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1#:~:text=The%202D%20convolution%20is%20a,into%20a%20single%20output%20pixel.) blog post. But anyways I'm attachin a small gif for your reference, with the actual equations! Happy coding.

![conv2d-gif](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*j-srXhSUW5s1Ysp86J6GHA.gif)

$$H_{out} = \frac{H_{in} - H_{filter}}{S} + 1$$ <br>
$$W_{out} = \frac{W_{in} - W_{filter}}{S} + 1$$ <br>
Where H, W stand for the height and width of the image and $H_{filter}$ or $W_{filter}$ denotes the filter size (or kernel size) and S is the stride. We have also added a batch_size dimension as you will see. **Note :  we are assuming there is no padding so don't worry about that.**

In [ ]:
import numpy as np
import pickle
import time
# Note : for all the following cells first you'll have to assign a few variables
# the number of appropriate lines does not include that! so dw if you overshoot
# them, the small documentation can be helpful in assigning names to variables

## Helper Function(s)

In [ ]:
np.random.seed(123) # Don't change this
with open('weights/out1.pkl', 'rb') as handle:
  out1 = pickle.load(handle)
with open('weights/out2.pkl', 'rb') as handle:
  out2 = pickle.load(handle)
with open('weights/out3.pkl', 'rb') as handle:
  out3 = pickle.load(handle)
x = np.random.randint(0, 255, (32, 3, 28, 28))
w = np.random.randint(-1, 1, (64, 3, 3, 3))
b = np.random.randint(-1, 1, (64, 1))
conv_param = {}
conv_param['stride'] = 1
input = {}
input["x"] = x
input["w"] = w
input["b"] = b
input["conv_param"] = conv_param

In [ ]:
class bcolors:
  OKGREEN = '\033[92m'
  WARNING = '\033[93m'
  ENDC = '\033[0m'

def function_checker(fn_name, input, output):
  '''
    Parameters
    ----------
    stmt - function to run
    input - predefined input
    output - same

    Returns
    -------
    Nothing! Just decides your extra credit xD.
    '''
  if fn_name == 'conv_forward_naive':
    assert (conv_forward_naive(input["x"], input["w"], input["b"], input["conv_param"]) == output).all(), f"{bcolors.WARNING}Wrong implementation!{bcolors.ENDC}"
    print(f"{bcolors.OKGREEN}Correct implementation.{bcolors.ENDC}")

  elif fn_name == 'stride_input':
    assert (stride_input(input["x"], input["w"], input["conv_param"]) == output).all(), f"{bcolors.WARNING}Wrong implementation!{bcolors.ENDC}"
    print(f"{bcolors.OKGREEN}Correct implementation.{bcolors.ENDC}")

  elif fn_name == 'conv_forward_vectorized':
    assert (conv_forward_vectorized(input["x"], input["w"], input["b"], input["conv_param"]) == output).all(), f"{bcolors.WARNING}Wrong implementation!{bcolors.ENDC}"
    assert (out1 == out3).all(), f"{bcolors.WARNING}Your outputs (naive and vectorized) don't match!{bcolors.ENDC}"
    print(f"{bcolors.OKGREEN}Correct implementation.{bcolors.ENDC}")

In [ ]:
def timeit(stmt, globals):
    '''
    Parameters
    ----------
    stmt - function to run
    globals - dictionary of current global variables

    Returns
    -------
    Nothing! Just prints relevant information about run-times.
    '''
    import timeit as _timeit
    import numpy as np

    # Rough approximation of a single run
    trial = _timeit.timeit(stmt, globals=globals, number=1)

    # Maximum duration
    duration = 1.0

    # Number of repeat
    repeat = 3

    # Compute rounded number of trials
    number = max(1,int(10**np.floor(np.log(duration/trial/repeat)/np.log(10))))

    # Only report best run
    best = min(_timeit.repeat(stmt, globals=globals, number=number, repeat=repeat))

    units = {"usec": 1, "msec": 1e3, "sec": 1e6}
    precision = 3
    usec = best * 1e6 / number
    if usec < 1000:
        print("%d loops, best of %d: %.*g usec per loop" % (number, repeat,
                                                            precision, usec))
    else:
        msec = usec / 1000
        if msec < 1000:
            print("%d loops, best of %d: %.*g msec per loop" % (number, repeat,
                                                                precision, msec))
        else:
            sec = msec / 1000
            print("%d loops, best of %d: %.*g sec per loop" % (number, repeat,
                                                               precision, sec))

## Naïve for-loop implementation

In [ ]:
def conv_forward_naive(x, w, b, conv_param):
    """
    A naive implementation of the forward pass for a convolutional layer.

    The input consists of N data points, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height HH and width HH.

    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, HH, WW)
    - b: Biases, of shape (F,)
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.

    Returns:
    - out: Output data, of shape (N, F, H', W') where H' and W' are given by
      H' = 1 + (H - HH) / stride
      W' = 1 + (W - WW) / stride
    """
    out = None
    ###########################################################################
    # TODO: Implement the convolutional forward pass. (our main solution has  #
    # 4 for loops apart from the assignment statements for various variables) #
    ###########################################################################

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return out

In [ ]:
start = time.time()
conv_forward_naive(x, w, b, conv_param)
print(f"Naive implementation took : {time.time() - start}s")
function_checker('conv_forward_naive', input, out1)

## Vectorized implementation

Warning : this is not easy! A rough sketch of what you have to do.

![method](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*whVcezzOdfMxDd5UZUG3tw.png)

First (the hard part), you'll have to manipulate your input to be in this modified shape, such that, it can be 'strided over' easily. Hint : [np.lib.stride_tricks.as_strided](https://numpy.org/doc/stable/reference/generated/numpy.lib.stride_tricks.as_strided.html)!

In [ ]:
def stride_input(x, w, conv_param):
    """
    Input manipulation before the vectorized Convolution.

    The input consists of N data points, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height HH and width HH.

    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, HH, WW)
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.

    Returns:
    - out: Output data, of shape (N, F, H', W', HH, WW) where H' and W' are given by
      H' = 1 + (H - HH) / stride
      W' = 1 + (W - WW) / stride
    """
    out = None
    ###########################################################################
    # TODO: Implement the manipulation as in the diagram.                     #
    # The main code is ~ 4 lines only!                                        #
    ###########################################################################

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return out

In [ ]:
function_checker('stride_input', input, out2)

Now (the easy part), you have to 'efficiently' sum over the new dimesnions/windows that you have just created. Hint : [np.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html).


In [ ]:
def conv_forward_vectorized(x, w, b, conv_param):
    """
    A vectorized implementation of the forward pass for a convolutional layer.

    The input consists of N data points, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height HH and width HH.

    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, HH, WW)
    - b: Biases, of shape (F,)
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.

    Returns:
    - out: Output data, of shape (N, F, H', W') where H' and W' are given by
      H' = 1 + (H - HH) / stride
      W' = 1 + (W - WW) / stride
    """
    out = None
    ###########################################################################
    # TODO: Implement the convolutional forward pass. (main code ~ 2 lines)   #
    # Hint: might have to use np.newaxis as well.                             #
    ###########################################################################

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return out

In [ ]:
start = time.time()
conv_forward_vectorized(x, w, b, conv_param)
print(f"Vectorized implementation took : {time.time() - start}s")
function_checker('conv_forward_vectorized', input, out3)

Ideally you should observe a huge speedup! (can be as high as 85x)

## Thank you. <br>
This small tutorial was made by [Karan Bania](https://karannb.github.io).<br>